# Palindrome Data Science Residency Assessment
Time taken = 2 hrs 50 mins. 

## Initial Data Exploration:
First I am just importing the packages that I am going to use. I then import the data, removing the header and ensuring that the columns names are correct. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [139]:
df = pd.read_csv("palindrome_data_test.csv", header=None)
df.drop([0], inplace = True)
df.columns = df.iloc[0].to_numpy()
df.drop([1], inplace = True)
correct_dtypes = {'Prevalence_%':'float32','Prevalence_LCL':'float32', 'Prevalence_UCL':'float32', 
                  'NoPLHIV': 'int32', 'NoPLHIV_LCL': 'int32', 'NoPLHIV_UCL': 'int32'}
df = df.astype(correct_dtypes)
print(df.dtypes)
df.reindex()
df.head()

District           object
Code               object
Estimate           object
Prevalence_%      float32
Prevalence_LCL    float32
Prevalence_UCL    float32
NoPLHIV             int32
NoPLHIV_LCL         int32
NoPLHIV_UCL         int32
dtype: object


,District,Code,Estimate,Prevalence_%,Prevalence_LCL,Prevalence_UCL,NoPLHIV,NoPLHIV_LCL,NoPLHIV_UCL
2,Buffalo City,BUF,Survey,13.600000,10.268000,16.931999,102437,77340,127534
3,Buffalo City,BUF,Fay-Heriott,13.501284,11.031605,16.421803,101693,83092,123691
4,City of Cape Town,CPT,Survey,5.200000,3.044000,7.356000,200751,117517,283985
5,City of Cape Town,CPT,Fay-Heriott,6.020042,4.335072,8.303091,232409,167359,320548
6,West Coast,DC1,Survey,2.700000,0.000000,5.444000,11302,0,22789


## 2a) What is the total number of people living with HIV in districts according to the Survey estimate?

In [140]:
total_est = df[df.Estimate == 'Survey'].NoPLHIV.sum()
total_lcl = df[df.Estimate == 'Survey'].NoPLHIV_LCL.sum()
total_ucl = df[df.Estimate == 'Survey'].NoPLHIV_UCL.sum()
print('total_est:', total_est, ', total_LCL:', total_lcl, ', total_UCL:', total_ucl)
average_dist = np.mean([total_ucl - total_est, total_est - total_lcl ])
print(average_dist)

total_est: 6409903 , total_LCL: 4235478 , total_UCL: 8589210
2176866.0


The total number of people of living with HIV in these districts according to Survey data is $6409903 \pm 2176866$, this is taking into account the estimates for the upper and lower condidence levels. 

## 2b) What is the average NoPLHIV of the two estimates for "Xhariep"?

In [141]:
df[df.District == 'Xhariep']

,District,Code,Estimate,Prevalence_%,Prevalence_LCL,Prevalence_UCL,NoPLHIV,NoPLHIV_LCL,NoPLHIV_UCL
18,Xhariep,DC16,Survey,8.30000,6.144000,10.456000,11777,8718,14836
19,Xhariep,DC16,Fay-Heriott,8.96307,7.187823,11.124214,12718,10199,15784


In [142]:
mean = df[df.District == 'Xhariep'].NoPLHIV.mean()
std = df[df.District == 'Xhariep'].NoPLHIV.std()
print('The average of the two estimates is', mean, 'with a standard deviation of', std)

The average of the two estimates is 12247.5 with a standard deviation of 665.3874810965413


## 2c) How many people are not living with HIV per district per survey?

We denote the number of people not living with HIV as NoPNLHIV. Taking into account the prevalance percentage and the number of people living with HIV per district we are able to determine that $NoPNLHIV = \frac{(100 - Prevalance)}{100} \times \frac{100 \times NoPLHIV}{Prevalance}$. 

In [154]:
NoPNLHIV= []
for i in range(2, len(df.index) + 2):
    a, b = df.loc[i,'Prevalence_%'], df.loc[i, 'NoPLHIV']
    NoPNLHIV.append(int(((100 - a) / 100.0) * ((100 * b)/ a)))

df['NoPNLHIV'] = NoPNLHIV
df.head()

,District,Code,Estimate,Prevalence_%,Prevalence_LCL,Prevalence_UCL,NoPLHIV,NoPLHIV_LCL,NoPLHIV_UCL,NoPNLHIV
2,Buffalo City,BUF,Survey,13.600000,10.268000,16.931999,102437,77340,127534,650776
3,Buffalo City,BUF,Fay-Heriott,13.501284,11.031605,16.421803,101693,83092,123691,651516
4,City of Cape Town,CPT,Survey,5.200000,3.044000,7.356000,200751,117517,283985,3659845
5,City of Cape Town,CPT,Fay-Heriott,6.020042,4.335072,8.303091,232409,167359,320548,3628178
6,West Coast,DC1,Survey,2.700000,0.000000,5.444000,11302,0,22789,407290


## 2d) What is the total NoPLHIV in all the cities?

In [160]:
df_city = df[df.District.str.contains("City")]  #Isolating City and Metro districts
df_city.append(df[df.District.str.contains("Metro")])

,District,Code,Estimate,Prevalence_%,Prevalence_LCL,Prevalence_UCL,NoPLHIV,NoPLHIV_LCL,NoPLHIV_UCL,NoPNLHIV
2,Buffalo City,BUF,Survey,13.600000,10.268000,16.931999,102437,77340,127534,650776
3,Buffalo City,BUF,Fay-Heriott,13.501284,11.031605,16.421803,101693,83092,123691,651516
4,City of Cape Town,CPT,Survey,5.200000,3.044000,7.356000,200751,117517,283985,3659845
5,City of Cape Town,CPT,Fay-Heriott,6.020042,4.335072,8.303091,232409,167359,320548,3628178
98,City of Johannesburg,JHB,Survey,11.100000,7.964000,14.236000,515387,369778,660996,4127738
99,City of Johannesburg,JHB,Fay-Heriott,11.075306,8.760104,13.909111,514241,406743,645818,4128890
104,City of Tshwane,TSH,Survey,11.700000,7.388000,16.011999,361450,228239,494661,2727866
105,City of Tshwane,TSH,Fay-Heriott,11.024858,8.255935,14.574909,340593,255052,450265,2748725
102,Nelson Mandela Bay Metro,NMA,Survey,8.300000,3.400000,13.200000,97459,39923,154995,1076745
103,Nelson Mandela Bay Metro,NMA,Fay-Heriott,9.054060,6.128703,13.179704,106313,71964,154757,1067889


In [169]:
# Determine HIV patients predicted per Survey
survey_est = df_city[df_city.Estimate == 'Survey'].NoPLHIV.sum() 
FH_est = df_city[df_city.Estimate == 'Fay-Heriott'].NoPLHIV.sum()

# Determine HIV patients predicted per Survey according to LCL and UCL
survey_est_LCL = df_city[df_city.Estimate == 'Survey'].NoPLHIV_LCL.sum() 
FH_est_LCL = df_city[df_city.Estimate == 'Fay-Heriott'].NoPLHIV_LCL.sum()

survey_est_UCL = df_city[df_city.Estimate == 'Survey'].NoPLHIV_UCL.sum()
FH_est_UCL = df_city[df_city.Estimate == 'Fay-Heriott'].NoPLHIV_UCL.sum()

# Taking the average of the two estimations methods 
np.mean([survey_est, FH_est]) 

1184480.5

In [167]:
df_per_survey = pd.DataFrame({'Estimate Type': ['Survey', 'Fay-Heriott'], 'Urban_NoPLHIV': [survey_est, FH_est], 
               'Urban_NoPLHIV_LCL': [survey_est_LCL, FH_est_LCL], 
                'Urban_NoPLHIV_UCL': [survey_est_UCL, FH_est_UCL]})
df_per_survey

,Estimate Type,Urban_NoPLHIV,Urban_NoPLHIV_LCL,Urban_NoPLHIV_UCL
0,Survey,1180025,792874,1567176
1,Fay-Heriott,1188936,912246,1540322


The total NoPLHIV who are living in city districts is $1184480$ (this was calculated as the average Urban_NoPLHIV of the two estimation methods). 

In [172]:
#writing to file
df.to_csv('palindrome_data_answer.csv', index=False)